In [85]:
import pytesseract
import cv2
from PIL import Image
import pandas as pd
import numpy as np

In [86]:
path = 'C:\\Users\\kakut\\OneDrive\\Desktop\\ocr images\\'
image_file = f'{path}photo_2024-02-20_16-52-48.jpg'
img = cv2.imread(image_file)

In [87]:
ocr_result  = pytesseract.image_to_string(img)
print(ocr_result)

Random

Ownership Random jwnershi
nme Marco's Pizza Store type number Marco's Pizza Store : type :
fi SO c 12 2040 Ottawa River Rd c
Cc 13 2116 N Reynolds Rd &
02 2652 W Central Av Cc 14 3678 Rugby Dr Cc
03 630 Dixie Hwy M 15 1419 South Av Cc
04 3510 Dorr St c 16 1234 W Sylvania Av c
05 5055 Glendale Av c 17. 4624 Woodville Rd M
06 3382 Lagrange St M 18 5155 S Main M
07 2525 W Laskey Rd Cc 19 106 E Airport Hwy c
08 303 Louisiana Av Cc 20 6725 W Central & McCord M
09 149 Main St Cc 21 4252 Monroe Cc
10 835 S McCord Rd M 22 2036 Woodville Rd Cc
“A 3501 Monroe St M 23 1316 Michigan Av M




In [88]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
cv2.imwrite(f"{path}\\temp\\index_gray.png", gray)


True

In [89]:
blur = cv2.GaussianBlur(gray, (7,7), 0)
cv2.imwrite(f"{path}temp\\index_blur.png", blur)

True

In [90]:
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
cv2.imwrite(f"{path}temp\\index_thresh.png", thresh)

True

In [91]:
kernal = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 20))
cv2.imwrite(f"{path}temp\\index_kernal.png", kernal)

True

In [92]:
dilate = cv2.dilate(thresh, kernal, iterations=1)
cv2.imwrite(f"{path}temp\\index_dilate.png", dilate)


True

In [108]:
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
cnts = sorted(cnts, key=lambda x: cv2.boundingRect(x)[0])

In [142]:
results = []
ownership = [] 
for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if h > 400 and w > 90:
        roi = img[y:y+h, x:x+h]
        cv2.imwrite("temp/index_roi.png", roi)
        cv2.rectangle(img, (x, y), (x+w, y+h), (36, 255, 12), 2)
        ocr_result = pytesseract.image_to_string(roi)
        ocr_result = ocr_result.split("\n")
        for item in ocr_result:
            if len(item) > 2:
                results.append(item)
cv2.imwrite(f"{path}temp\\index_bbox_new.png", img)

True

In [148]:
filtered_result = [x for x in results if  x[0].isdigit()]
df = pd.DataFrame({'Store':filtered_result})
df = df.drop_duplicates(subset=['Store'])


In [149]:
df = df.reset_index(drop=True)
df

,Store
0,2607 Starr Av
1,309 W Alexis Rd
2,2652 W Central Av
3,630 Dixie Hwy
4,3510 Dorr St
5,5055 Glendale Av
6,3382 Lagrange St
7,2525 W Laskey Rd
8,303 Louisiana Av
9,149 Main St


In [150]:
df = df.drop([12,13,14])
df = df.reset_index(drop=True)

,Store
0,2607 Starr Av
1,309 W Alexis Rd
2,2652 W Central Av
3,630 Dixie Hwy
4,3510 Dorr St
5,5055 Glendale Av
6,3382 Lagrange St
7,2525 W Laskey Rd
8,303 Louisiana Av
9,149 Main St


In [153]:
random_stores = df.sample(n=3, random_state=42)  # Set random_state for reproducibility
random_stores

,Store
8,303 Louisiana Av
16,1234 W Sylvania Av
0,2607 Starr Av


In [154]:
start_row = 3
step = 7
systematic_sample = df.iloc[start_row::step]
systematic_sample

,Store
3,630 Dixie Hwy
10,835 S McCord Rd
17,4624 Woodville Rd
